<a href="https://colab.research.google.com/github/harnalashok/deeplearning/blob/main/John_Labs_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/NER_EN.ipynb)




# **Sentiment Prediction in English text**
Refer [here](https://nlp.johnsnowlabs.com/docs/en/annotators#classifierdl) <br>
For appropriate spark-nlp models and pipeline make a search [here](https://nlp.johnsnowlabs.com/models)

Trains a `ClassifierDL` for generic Multi-class Text Classification. 

`ClassifierDL` uses the state-of-the-art `Universal Sentence Encoder` as an input for text classifications. The `ClassifierDL` annotator uses a deep learning model (DNNs) built inside TensorFlow and supports up to 100 classes.

For instantiated/pretrained models, see ClassifierDLModel.

Note: This annotator accepts a label column of a single item in either type of String, Int, Float, or Double. UniversalSentenceEncoder, BertSentenceEmbeddings, or SentenceEmbeddings can be used for the inputCol

## 1. Colab Setup
Install spark-nlp

In [1]:
# Installs pyspark, spark-nlp and findspark

!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

# !bash colab.sh
# -p is for pyspark
# -s is for spark-nlp
# !bash colab.sh -p 3.1.1 -s 3.0.1
# by default they are set to the latest



--2021-11-11 23:45:35--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://setup.johnsnowlabs.com/colab.sh [following]
--2021-11-11 23:45:37--  https://setup.johnsnowlabs.com/colab.sh
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2021-11-11 23:45:38--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:44

In [2]:
# To check contents of colab.sh, just download 
#  colab.sh and examine its contents:

!wget http://setup.johnsnowlabs.com/colab.sh 

! cat /content/colab.sh


--2021-11-11 23:46:19--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://setup.johnsnowlabs.com/colab.sh [following]
--2021-11-11 23:46:20--  https://setup.johnsnowlabs.com/colab.sh
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2021-11-11 23:46:21--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:44

In [3]:
# Install Spark NLP Display for visualization
# !pip install --ignore-installed spark-nlp

## 2. Start the Spark session

Import dependencies and start Spark session.

In [34]:
import pandas as pd
import numpy as np
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import DoubleType
#Replace part of string with another string
from pyspark.sql.functions import regexp_replace



In [6]:
# Create Spark session
# And start sparknlp

spark = sparknlp.start()

In [7]:
# Show multiple command outputs from a cell

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [8]:
# Mount gdrive to read data
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


## 3. Read data

In [10]:
path = "/gdrive/MyDrive/Colab_data_files/corona_nlp/Corona_NLP_test.csv"

In this example, the training data `"sentiment.csv"` has the form of --text,label--. Examples:
```
This movie is the best movie I have wached ever! In my opinion this movie can win an award.,0
This was a terrible movie! The acting was bad really bad!,1
```

In [17]:
df = spark.read.csv(path=path, inferSchema=True, header=True)

In [ ]:
df.show(40)
df.count()   # 6792

In [19]:
df = df.dropna()

In [20]:
df.count()  # 1944

1944

In [21]:
df.columns

['UserName', 'ScreenName', 'Location', 'TweetAt', 'OriginalTweet', 'Sentiment']

In [23]:
df = df.select('OriginalTweet', 'Sentiment')

In [24]:
df.show()

+--------------------+--------------------+
|       OriginalTweet|           Sentiment|
+--------------------+--------------------+
|TRENDING: New Yor...|  Extremely Negative|
|When I couldn't f...|            Positive|
|#toiletpaper #dun...|             Neutral|
|Do you remember t...|             Neutral|
|Anyone been in a ...|  Extremely Positive|
|While we were bus...|            Positive|
|When youre stock...|             Neutral|
|#CoronaVirus #COV...|            Negative|
|"For those of you...|  Extremely Positive|
|Control over stoc...|  Extremely Positive|
|When your usual g...|            Negative|
|"#Coronavirus is ...|"" says NELP Gove...|
|Two important pho...|            Negative|
|IMHO Russia's rug...|  Extremely Negative|
|So not due to #Br...|             Neutral|
|Fellow Uni instru...|            Negative|
|1/25 Part 1 - Why...|            Negative|
|Thread: Please ST...|  Extremely Positive|
|For those thinkin...|            Positive|
|Global stock mark...|          

In [27]:
# Read dataset using pandas
f = pd.read_csv(path)
f.head()
f['Sentiment'].value_counts()


,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,1,44953,NYC,02-03-2020,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,2,44954,"Seattle, WA",02-03-2020,When I couldn't find hand sanitizer at Fred Me...,Positive
2,3,44955,NaN,02-03-2020,Find out how you can protect yourself and love...,Extremely Positive
3,4,44956,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious sh...,Negative
4,5,44957,"Melbourne, Victoria",03-03-2020,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral


Negative              1041
Positive               947
Neutral                619
Extremely Positive     599
Extremely Negative     592
Name: Sentiment, dtype: int64

In [28]:
g = f[['OriginalTweet','Sentiment']].copy()

In [29]:
smallCorpus =spark.createDataFrame(g)

In [30]:
smallCorpus.show()

+--------------------+------------------+
|       OriginalTweet|         Sentiment|
+--------------------+------------------+
|TRENDING: New Yor...|Extremely Negative|
|When I couldn't f...|          Positive|
|Find out how you ...|Extremely Positive|
|#Panic buying hit...|          Negative|
|#toiletpaper #dun...|           Neutral|
|Do you remember t...|           Neutral|
|Voting in the age...|          Positive|
|@DrTedros "We can...|           Neutral|
|HI TWITTER! I am ...|Extremely Negative|
|Anyone been in a ...|Extremely Positive|
|Best quality couc...|          Positive|
|Beware of counter...|Extremely Negative|
|Panic food buying...|Extremely Negative|
|#Covid_19 Went to...|Extremely Positive|
|While we were bus...|          Positive|


@f...|Extremely Negative|
|What Precautionar...|Extremely Positive|
|When youre stock...|           Neutral|
|That's about a we...|          Positive|
|Studies show the ...|Extremely Positive|
+--------------------+------------------+
only s

In [31]:
smallCorpus.count()
smallCorpus = smallCorpus.dropna()

3798

In [32]:
smallCorpus.count()

3798

In [ ]:
smallCorpus.

In [38]:
#Replace part of string with another string
smallCorpus.withColumn('Sentiment', regexp_replace('Sentiment', 'Negative', ))
smallCorpus.show()

Py4JError: ignored

In [40]:
# Change data type of 'label' column
smallCorpus = smallCorpus.withColumn(
                                      "label",
                                      smallCorpus["Sentiment"]
                                    )

smallCorpus = smallCorpus.withColumn(
                                      "text",
                                      smallCorpus["OriginalTweet"]
                                    )

smallCorpus.show()

+--------------------+------------------+------------------+--------------------+
|       OriginalTweet|         Sentiment|             label|                text|
+--------------------+------------------+------------------+--------------------+
|TRENDING: New Yor...|Extremely Negative|Extremely Negative|TRENDING: New Yor...|
|When I couldn't f...|          Positive|          Positive|When I couldn't f...|
|Find out how you ...|Extremely Positive|Extremely Positive|Find out how you ...|
|#Panic buying hit...|          Negative|          Negative|#Panic buying hit...|
|#toiletpaper #dun...|           Neutral|           Neutral|#toiletpaper #dun...|
|Do you remember t...|           Neutral|           Neutral|Do you remember t...|
|Voting in the age...|          Positive|          Positive|Voting in the age...|
|@DrTedros "We can...|           Neutral|           Neutral|@DrTedros "We can...|
|HI TWITTER! I am ...|Extremely Negative|Extremely Negative|HI TWITTER! I am ...|
|Anyone been in 

In [ ]:
# Change data type of 'label' column
smallCorpus = smallCorpus.withColumn(
                                      "label",
                                      smallCorpus["label"].cast(DoubleType()
                                    )

In [ ]:
#smallCorpus = spark.read.option("header","True").csv("/content/combined.csv")
#smallCorpus = spark.read.csv(path = "/content/combined.csv",
#                             header = True,
#                             inferSchema = True)



In [41]:

documentAssembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

In [42]:


useEmbeddings = UniversalSentenceEncoder.pretrained() \
    .setInputCols("document") \
    .setOutputCol("sentence_embeddings")

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]


In [43]:
docClassifier = ClassifierDLApproach() \
    .setInputCols("sentence_embeddings") \
    .setOutputCol("category") \
    .setLabelColumn("label") \
    .setBatchSize(64) \
    .setMaxEpochs(20) \
    .setLr(5e-3) \
    .setDropout(0.5)

In [44]:

pipeline = Pipeline() \
    .setStages(
      [
        documentAssembler,
        useEmbeddings,
        docClassifier
      ]
    )




In [45]:
pipelineModel = pipeline.fit(smallCorpus)

In [51]:
st = smallCorpus.select('text')

In [52]:
pred = pipelineModel.transform(st)

In [53]:
pred.columns

['text', 'document', 'sentence_embeddings', 'category']

In [56]:
pred.select('category').show(truncate = False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|category                                                                                                                                                                                                  |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[[category, 0, 227, Negative, [Neutral -> 0.012181358, Extremely Negative -> 3.2265532E-8, Negative -> 0.9848771, Positive -> 0.0029413248, sentence -> 0, Extremely Positive -> 1.972186E-7], []]]       |
|[[category, 0, 192, Positive, [Neutral -> 0.0022905956, Extremely Negative -> 2.5735807E-9, Negative -> 0.020018954, Positive -> 0.97753006, sentence -> 0, Extremely Positive -> 1

In [57]:
p = pred.toPandas()

In [58]:
p.head()

,text,document,sentence_embeddings,category
0,TRENDING: New Yorkers encounter empty supermar...,"[(document, 0, 227, TRENDING: New Yorkers enco...","[(sentence_embeddings, 0, 227, TRENDING: New Y...","[(category, 0, 227, Negative, {'sentence': '0'..."
1,When I couldn't find hand sanitizer at Fred Me...,"[(document, 0, 192, When I couldn't find hand ...","[(sentence_embeddings, 0, 192, When I couldn't...","[(category, 0, 192, Positive, {'sentence': '0'..."
2,Find out how you can protect yourself and love...,"[(document, 0, 72, Find out how you can protec...","[(sentence_embeddings, 0, 72, Find out how you...","[(category, 0, 72, Extremely Positive, {'sente..."
3,#Panic buying hits #NewYork City as anxious sh...,"[(document, 0, 317, #Panic buying hits #NewYor...","[(sentence_embeddings, 0, 317, #Panic buying h...","[(category, 0, 317, Negative, {'sentence': '0'..."
4,#toiletpaper #dunnypaper #coronavirus #coronav...,"[(document, 0, 251, #toiletpaper #dunnypaper #...","[(sentence_embeddings, 0, 251, #toiletpaper #d...","[(category, 0, 251, Neutral, {'sentence': '0',..."


In [ ]:
# If you change the model, re-run all the cells below
# Other applicable models: ner_dl, ner_dl_bert
MODEL_NAME = "onto_100"

## 4. Some sample examples

In [ ]:
text_list = [
    """William Henry Gates III (born October 28, 1955) is an American business magnate, software developer, investor, and philanthropist. He is best known as the co-founder of Microsoft Corporation. During his career at Microsoft, Gates held the positions of chairman, chief executive officer (CEO), president and chief software architect, while also being the largest individual shareholder until May 2014. He is one of the best-known entrepreneurs and pioneers of the microcomputer revolution of the 1970s and 1980s. Born and raised in Seattle, Washington, Gates co-founded Microsoft with childhood friend Paul Allen in 1975, in Albuquerque, New Mexico; it went on to become the world's largest personal computer software company. Gates led the company as chairman and CEO until stepping down as CEO in January 2000, but he remained chairman and became chief software architect. During the late 1990s, Gates had been criticized for his business tactics, which have been considered anti-competitive. This opinion has been upheld by numerous court rulings. In June 2006, Gates announced that he would be transitioning to a part-time role at Microsoft and full-time work at the Bill & Melinda Gates Foundation, the private charitable foundation that he and his wife, Melinda Gates, established in 2000.[9] He gradually transferred his duties to Ray Ozzie and Craig Mundie. He stepped down as chairman of Microsoft in February 2014 and assumed a new post as technology adviser to support the newly appointed CEO Satya Nadella.""",
    """The Mona Lisa is a 16th century oil painting created by Leonardo. It's held at the Louvre in Paris."""
]

## 5. Define Spark NLP pipeline

In [ ]:
documentAssembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

# ner_dl and onto_100 model are trained with glove_100d, so the embeddings in
# the pipeline should match
if (MODEL_NAME == "ner_dl") or (MODEL_NAME == "onto_100"):
    embeddings = WordEmbeddingsModel.pretrained('glove_100d') \
        .setInputCols(["document", 'token']) \
        .setOutputCol("embeddings")

# Bert model uses Bert embeddings
elif MODEL_NAME == "ner_dl_bert":
    embeddings = BertEmbeddings.pretrained(name='bert_base_cased', lang='en') \
        .setInputCols(['document', 'token']) \
        .setOutputCol('embeddings')

ner_model = NerDLModel.pretrained(MODEL_NAME, 'en') \
    .setInputCols(['document', 'token', 'embeddings']) \
    .setOutputCol('ner')

ner_converter = NerConverter() \
    .setInputCols(['document', 'token', 'ner']) \
    .setOutputCol('ner_chunk')

nlp_pipeline = Pipeline(stages=[
    documentAssembler, 
    tokenizer,
    embeddings,
    ner_model,
    ner_converter
])

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]
onto_100 download started this may take some time.
Approximate size to download 13.5 MB
[OK!]


## 6. Run the pipeline

In [ ]:
empty_df = spark.createDataFrame([['']]).toDF('text')
pipeline_model = nlp_pipeline.fit(empty_df)
df = spark.createDataFrame(pd.DataFrame({'text': text_list}))
result = pipeline_model.transform(df)

## 7. Visualize results

In [ ]:
from sparknlp_display import NerVisualizer

NerVisualizer().display(
    result = result.collect()[0],
    label_col = 'ner_chunk',
    document_col = 'document'
)